In [ ]:
import glob
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'  # 也可以使用 tensorflow
# os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,exception_verbosity=high'
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cuda,optimizer=fast_compile'
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import keras.backend as K

K.set_image_data_format('channels_last')

import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, LSTM, Concatenate, Dense, Dropout
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten, UpSampling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.datasets import mnist
from urllib.request import urlretrieve
from keras.optimizers import RMSprop, SGD, Adam, sgd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from keras.models import model_from_json

# from read_data import *
a = %pwd
print(a)

%run /root/jupyter/inspace/sang-min/myo_proejct/load_data.py import DataLoader
#%load load_data import DataLoader
#from load_data import DataLoader


'''
Model structure

Input : (100) Vector
Output : (64, 64, 1) image

Generator
100 -> 256
256 -> 16 x 16
16 x 16 -> 32 x 32
32 x 32 -> 64 x 64
64 x 64 -> 128 x 128

Discriminator
128 x 128 -> 64 x 64
64 x 64 -> 32 x 32
32 x 32 -> 16 x 16
16 x 16 -> 8 x 8
8 x 8 -> 64
64 -> 32 (16, 10)

'''
loader = DataLoader(data_path='./dataset_2018_05_16/', is_real_image=False)

conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02)

lstm_size = (300, 16)
noise_size = 100
image_size = 128
input_size = 100
image_channel = 1
learning_rate = 2e-4

def load_model():
    json_file = open('./model2_output/load_model/g_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    load_g_model = model_from_json(loaded_model_json)
    # load weights into new model
    load_g_model.load_weights("./model2_output/load_model/g_model.h5")
    
    json_file = open('./model2_output/load_model/d_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    load_d_model = model_from_json(loaded_model_json)
    # load weights into new model
    load_d_model.load_weights("./model2_output/load_model/d_model.h5")
    
    json_file = open('./model2_output/load_model/combined_model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    load_combined_model = model_from_json(loaded_model_json)
    # load weights into new model
    load_combined_model.load_weights("./model2_output/load_model/combined_model.h5")
    
    print("Loaded model from disk")
    
    return [load_g_model, load_d_model, load_combined_model]

def sample_generation(num, net_g):
    for _ in range(num):
        gan_image = net_g.predict(noise)
        cv2.imwrite('./model2_output/image/' + 'sample image'+ str(_) + '.png', gan_image[_] * 127.5)
        
    print("generated image")


def generative_model(noise_size, image_channel):
    
    print(" _ : ", noise_input)
    _ = Dense(256, input_shape=(100,), activation='relu')(noise_input)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Reshape((16, 16, 1), input_shape=(256,))(_)

    _ = Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer=conv_init, input_shape=(16, 16, 1))(
        _)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Activation(activation='relu')(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=256, kernel_size=3, padding='same', kernel_initializer=conv_init,
                        input_shape=(16, 16, 128))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Activation(activation='relu')(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=512, kernel_size=3, padding='same',  kernel_initializer=conv_init,
                        input_shape=(32, 32, 256))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Activation(activation='relu')(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=256, kernel_size=3, padding='same', kernel_initializer=conv_init,
                        input_shape=(64, 64, 512))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Activation(activation='relu')(_)

    #_ = UpSampling2D()(_)
    _ = Conv2D(filters=1, kernel_size=3, padding='same', kernel_initializer=conv_init,
               input_shape=(128, 128, 256))(_)
    _ = Activation(activation='tanh')(_)

    return Model(inputs=noise_input, outputs=_)


def discriminative_model(image_size, image_channel):
    _ = inputs = Input(shape=(image_size, image_size, image_channel))

    _ = Conv2D(filters=256, kernel_size=(2, 2), strides=2, padding='same', input_shape=(128, 128, 1),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=512, kernel_size=(1, 1), strides=2, padding='same', input_shape=(64, 64, 256),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=256, kernel_size=(2, 2), strides=2, padding='same', input_shape=(32, 32, 512),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=128, kernel_size=(2, 2), strides=2, padding='same', input_shape=(16, 16, 256),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=128, kernel_size=(2, 2), strides=2, padding='same', input_shape=(8, 8, 128),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=image_channel, kernel_size=(2, 2), strides=1, padding='same', input_shape=(4, 4, 128),
               kernel_initializer=conv_init)(_)
    # _ = LeakyReLU(alpha=0.2)(_)

    outputs = Flatten()(_)
    outputs = Dense(1, activation='sigmoid')(outputs)

    return Model(inputs=inputs, outputs=outputs)

discriminator_optim = sgd(lr=0.01, momentum=0.9, nesterov=True)
generator_optim = Adam(lr=1e-3)
adam = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)  # as described in appendix A of DeepMind's AC-GAN paper

# lstm_input = Input(shape=lstm_size)
noise_input = Input(shape=(noise_size,))
real_image = Input(shape=(image_size, image_size, image_channel))

net_g = generative_model(noise_size, image_channel)
fake_image = net_g(noise_input)
net_g.summary()

net_d = discriminative_model(image_size, image_channel)
net_d.summary()

combined_output = net_d(fake_image)
combined_model = Model(inputs=[noise_input], outputs=[combined_output], name='combined')

'''
net_g, net_d, combined_model = load_model()

net_g.summary()
net_d.summary()

fake_image = net_g(noise_input)
'''

net_g.compile(loss='binary_crossentropy', optimizer=adam)
net_d.compile(loss='binary_crossentropy', optimizer=adam)
net_d.trainable = False
combined_model.compile(loss='binary_crossentropy', optimizer=adam)

combined_model.summary()

'''
loader = DataLoader(data_path='./MYO_Dataset_label/')

emg = loader.load_emg_data()
image, label = loader.load_image()

print(emg.shape)
print(image.shape, label)

emg = loader.get_emg_datas(10)
images, labels = loader.get_images(10)
print(emg.shape, images.shape, labels.shape)

'''

epoch = 30000
i = 0
time_0 = time.time()
batch_size = 64

d_step = 1
g_step = 2

g_loss_history = []
d_loss_real_history = []
d_loss_fake_history = []

while i <= epoch:
    # x_train = loader.get_emg_datas(batch_size)
    images = loader.get_images(batch_size)
    
    for _ in range(d_step):
        noise = np.random.normal(size=[batch_size, noise_size])

        g_z = net_g.predict(noise)
    
        d_loss_real = net_d.train_on_batch(images, np.random.uniform(low=0.7, high=1.2, size=batch_size))
        d_loss_fake = net_d.train_on_batch(g_z, np.random.uniform(low=0.0, high=0.3, size=batch_size))
    
        d_loss_real_history.append(d_loss_real)
        d_loss_fake_history.append(d_loss_fake)
        d_loss = np.sum([d_loss_fake, d_loss_real])    
        
    for _ in range(g_step):
        noise = np.random.normal(size=[batch_size, noise_size])
        combined_loss = combined_model.train_on_batch(noise, np.random.uniform(low=0.7, high=1.2, size=batch_size))
    
        g_loss_history.append(combined_loss)
    
    print("%d [D loss real: %f] [D loss fake: %f] [D loss: %f] [G loss: %f]" % (i, d_loss_real, d_loss_fake, d_loss, combined_loss))
    
    if i % 500 == 0:
        gan_image = net_g.predict(noise)
        print("gan imaga2 : ", gan_image[0].shape)
        cv2.imwrite('./model2_output/image/' + 'fake_image'+ str(i) + '.png', gan_image[0] * 127.5)
        #cv2.imwrite('./output_image3/' + 'real_image'+ str(i) + '.png', images[0] * 127.5)
        
    i +=1;

sample_generation(32, net_g)

net_g.save_weights("./model2_output/save_model/g_model.h5")
net_d.save_weights("./model2_output/save_model/d_model.h5")
combined_model.save_weights("./model2_output/save_model/combined_model.h5")

g_model_json = net_g.to_json()
with open("./model2_output/save_model/g_model.json", "w") as json_file:
    json_file.write(g_model_json)
    
d_model_json = net_d.to_json()
with open("./model2_output/save_model.json", "w") as json_file:
    json_file.write(d_model_json)

combined_model_json = combined_model.to_json()
with open("./model2_output/save_model/combined_model.json", "w") as json_file:
    json_file.write(combined_model_json)

print("Saved model to disk")

#print("save model ")
plt.figure(1, figsize=(16, 8))
plt.plot(d_loss_real_history)
plt.ylabel('d_loss_real')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./model2_output/image/d_loss_real_history.png')

plt.figure(2, figsize=(16, 8))
plt.plot(d_loss_fake_history)
plt.ylabel('d_loss_fake')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./model2_output/image/d_loss_fake_history.png')

#plt.show()

plt.figure(3, figsize=(16, 8))
plt.plot(g_loss_history)
plt.ylabel('g_loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./model2_output/image/g_loss_history.png')

#plt.show()\

plt.show()

print("Finish")

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


/root/jupyter/inspace/sang-min/myo_proejct
30
./dataset_2018_05_16/1/
15
 _ :  Tensor("input_1:0", shape=(?, 100), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       1280      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       64        
____________

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss real: 0.527479] [D loss fake: 1.236732] [D loss: 1.764211] [G loss: 0.263870]
gan imaga2 :  (128, 128, 1)
1 [D loss real: 0.222796] [D loss fake: 1.025529] [D loss: 1.248324] [G loss: 1.105755]
2 [D loss real: 0.899247] [D loss fake: 0.694668] [D loss: 1.593915] [G loss: 0.762267]
3 [D loss real: 0.356593] [D loss fake: 0.768312] [D loss: 1.124904] [G loss: 0.759786]
4 [D loss real: 0.331268] [D loss fake: 0.700129] [D loss: 1.031397] [G loss: 0.938569]
5 [D loss real: 0.262230] [D loss fake: 0.593229] [D loss: 0.855458] [G loss: 1.215372]
6 [D loss real: 0.253142] [D loss fake: 0.507868] [D loss: 0.761010] [G loss: 1.541951]
7 [D loss real: 0.219430] [D loss fake: 0.432243] [D loss: 0.651673] [G loss: 1.797926]
8 [D loss real: 0.245654] [D loss fake: 0.583528] [D loss: 0.829182] [G loss: 1.896900]
9 [D loss real: 0.209400] [D loss fake: 0.457878] [D loss: 0.667278] [G loss: 1.757779]
10 [D loss real: 0.241976] [D loss fake: 0.426825] [D loss: 0.668801] [G loss: 1.667318]
11 

91 [D loss real: 0.303267] [D loss fake: 0.432759] [D loss: 0.736026] [G loss: 1.732378]
92 [D loss real: 0.216252] [D loss fake: 0.441363] [D loss: 0.657615] [G loss: 1.805611]
93 [D loss real: 0.212306] [D loss fake: 0.452256] [D loss: 0.664562] [G loss: 1.771991]
94 [D loss real: 0.293127] [D loss fake: 0.398093] [D loss: 0.691220] [G loss: 1.758402]
95 [D loss real: 0.225910] [D loss fake: 0.411522] [D loss: 0.637432] [G loss: 1.766221]
96 [D loss real: 0.167217] [D loss fake: 0.404599] [D loss: 0.571816] [G loss: 1.913761]
97 [D loss real: 0.137859] [D loss fake: 0.405776] [D loss: 0.543635] [G loss: 1.880597]
./dataset_2018_05_16/8/
98 [D loss real: 0.181441] [D loss fake: 0.421755] [D loss: 0.603196] [G loss: 1.921171]
99 [D loss real: 0.276074] [D loss fake: 0.418490] [D loss: 0.694564] [G loss: 1.802567]
100 [D loss real: 0.080083] [D loss fake: 0.430542] [D loss: 0.510625] [G loss: 1.777424]
101 [D loss real: 0.146939] [D loss fake: 0.439609] [D loss: 0.586548] [G loss: 1.744

...

17515 [D loss real: 0.213626] [D loss fake: 0.416027] [D loss: 0.629652] [G loss: 1.856841]
17516 [D loss real: 0.189318] [D loss fake: 0.424445] [D loss: 0.613763] [G loss: 1.856392]
17517 [D loss real: 0.255767] [D loss fake: 0.410914] [D loss: 0.666681] [G loss: 1.779838]
17518 [D loss real: 0.135861] [D loss fake: 0.406530] [D loss: 0.542391] [G loss: 1.890013]
17519 [D loss real: 0.299124] [D loss fake: 0.479443] [D loss: 0.778567] [G loss: 1.799962]
17520 [D loss real: 0.084198] [D loss fake: 0.429151] [D loss: 0.513349] [G loss: 1.801439]
17521 [D loss real: 0.125081] [D loss fake: 0.399809] [D loss: 0.524890] [G loss: 1.783790]
./dataset_2018_05_16/3/
17522 [D loss real: 0.163991] [D loss fake: 0.406883] [D loss: 0.570874] [G loss: 1.779959]
17523 [D loss real: 0.223136] [D loss fake: 0.434098] [D loss: 0.657234] [G loss: 1.833532]
17524 [D loss real: 0.219334] [D loss fake: 0.421690] [D loss: 0.641025] [G loss: 1.820657]
17525 [D loss real: 0.198724] [D loss fake: 0.423446] [D

17603 [D loss real: 0.148094] [D loss fake: 0.421736] [D loss: 0.569830] [G loss: 1.800292]
17604 [D loss real: 0.184136] [D loss fake: 0.458240] [D loss: 0.642376] [G loss: 1.830896]
17605 [D loss real: 0.249212] [D loss fake: 0.415306] [D loss: 0.664518] [G loss: 1.845155]
17606 [D loss real: 0.232891] [D loss fake: 0.407529] [D loss: 0.640419] [G loss: 1.855231]
./dataset_2018_05_16/9/
17607 [D loss real: 0.227149] [D loss fake: 0.389144] [D loss: 0.616292] [G loss: 1.830783]
17608 [D loss real: 0.172526] [D loss fake: 0.417069] [D loss: 0.589595] [G loss: 1.804852]
17609 [D loss real: 0.165586] [D loss fake: 0.466292] [D loss: 0.631877] [G loss: 1.838049]
17610 [D loss real: 0.215785] [D loss fake: 0.434789] [D loss: 0.650573] [G loss: 1.834294]
17611 [D loss real: 0.230254] [D loss fake: 0.431657] [D loss: 0.661912] [G loss: 1.808629]
17612 [D loss real: 0.131064] [D loss fake: 0.439996] [D loss: 0.571060] [G loss: 1.804710]
17613 [D loss real: 0.175691] [D loss fake: 0.411933] [D

17691 [D loss real: 0.185766] [D loss fake: 0.405990] [D loss: 0.591756] [G loss: 1.794582]
17692 [D loss real: 0.189923] [D loss fake: 0.419750] [D loss: 0.609673] [G loss: 1.837988]
17693 [D loss real: 0.187527] [D loss fake: 0.396502] [D loss: 0.584029] [G loss: 1.883586]
17694 [D loss real: 0.171409] [D loss fake: 0.410513] [D loss: 0.581922] [G loss: 1.849198]
17695 [D loss real: 0.182962] [D loss fake: 0.466098] [D loss: 0.649060] [G loss: 1.832010]
17696 [D loss real: 0.282359] [D loss fake: 0.419458] [D loss: 0.701817] [G loss: 1.873667]
17697 [D loss real: 0.273358] [D loss fake: 0.423018] [D loss: 0.696377] [G loss: 1.765947]
17698 [D loss real: 0.163572] [D loss fake: 0.414681] [D loss: 0.578253] [G loss: 1.808682]
17699 [D loss real: 0.207119] [D loss fake: 0.434817] [D loss: 0.641935] [G loss: 1.778762]
17700 [D loss real: 0.311930] [D loss fake: 0.411039] [D loss: 0.722969] [G loss: 1.812537]
17701 [D loss real: 0.210344] [D loss fake: 0.443354] [D loss: 0.653698] [G loss

17779 [D loss real: 0.214670] [D loss fake: 0.402837] [D loss: 0.617507] [G loss: 1.778039]
17780 [D loss real: 0.214081] [D loss fake: 0.409964] [D loss: 0.624046] [G loss: 1.879212]
17781 [D loss real: 0.124807] [D loss fake: 0.381394] [D loss: 0.506202] [G loss: 1.865563]
17782 [D loss real: 0.199093] [D loss fake: 0.417773] [D loss: 0.616866] [G loss: 1.826781]
17783 [D loss real: 0.324356] [D loss fake: 0.430534] [D loss: 0.754891] [G loss: 1.825597]
17784 [D loss real: 0.173308] [D loss fake: 0.423441] [D loss: 0.596749] [G loss: 1.918728]
17785 [D loss real: 0.323622] [D loss fake: 0.375266] [D loss: 0.698888] [G loss: 1.839639]
17786 [D loss real: 0.194222] [D loss fake: 0.476290] [D loss: 0.670512] [G loss: 1.803515]
17787 [D loss real: 0.218721] [D loss fake: 0.446169] [D loss: 0.664890] [G loss: 1.830631]
17788 [D loss real: 0.163420] [D loss fake: 0.379201] [D loss: 0.542620] [G loss: 1.797523]
./dataset_2018_05_16/7/
17789 [D loss real: 0.243691] [D loss fake: 0.413351] [D

17867 [D loss real: 0.220274] [D loss fake: 0.424365] [D loss: 0.644639] [G loss: 1.748387]
17868 [D loss real: 0.155873] [D loss fake: 0.451490] [D loss: 0.607363] [G loss: 1.813164]
17869 [D loss real: 0.204810] [D loss fake: 0.431785] [D loss: 0.636595] [G loss: 1.772069]
17870 [D loss real: 0.189890] [D loss fake: 0.457363] [D loss: 0.647253] [G loss: 1.757100]
17871 [D loss real: 0.243389] [D loss fake: 0.383505] [D loss: 0.626894] [G loss: 1.767593]
17872 [D loss real: 0.199267] [D loss fake: 0.425915] [D loss: 0.625181] [G loss: 1.801804]
17873 [D loss real: 0.229558] [D loss fake: 0.442130] [D loss: 0.671688] [G loss: 1.818130]
./dataset_2018_05_16/13/
17874 [D loss real: 0.161221] [D loss fake: 0.462599] [D loss: 0.623820] [G loss: 1.765269]
17875 [D loss real: 0.193871] [D loss fake: 0.453070] [D loss: 0.646941] [G loss: 1.728892]
17876 [D loss real: 0.075997] [D loss fake: 0.417320] [D loss: 0.493317] [G loss: 1.728555]
17877 [D loss real: 0.218738] [D loss fake: 0.444153] [